In [1]:
import os
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
import pickle
from collections import defaultdict
import importlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

from themachinethatgoesping.echosounders import kongsbergall, simradraw, index_functions
from themachinethatgoesping import pingprocessing as pp
from themachinethatgoesping import tools as ptools

In [2]:
import themachinethatgoesping as tmtgp
tmtgp.version()

themachinethatgoesping
- version:       0.25.0

modules:
- tools_cppy     0.27.0
- tools          @PROJECT_VERSION@
- scripts        @PROJECT_VERSION@
- algorithms     0.8.2
- navigation     0.17.0
- echosounders_cppy 0.42.1
- echosounders   0.42.1
- pingprocessing_cppy 0.8.5
- pingprocessing @PROJECT_VERSION@
- gridding       @PROJECT_VERSION@


In [3]:
#create the output path
path_out = "../unittest_data/"
path_in = "./data_in/"

os.makedirs(path_out,exist_ok=True)
os.makedirs(path_in,exist_ok=True)

## Explore and sort input data

In [4]:
folders = index_functions.find_folders_with_files(path_in, [".all",".wcd", '.raw'], followlinks=True)
folders.sort()
N = 10

for input_path in folders:
    print(input_path)

Found 35 files
./data_in/kongsberg/a/c
./data_in/kongsberg/a/f/ALL
./data_in/kongsberg/a/f/WCD
./data_in/kongsberg/a/y
./data_in/kongsberg/e
./data_in/kongsberg/g
./data_in/kongsberg/he
./data_in/kongsberg/simon
./data_in/kongsberg/turbeams
./data_in/simrad/ek80_test/ek80_example_data_01
./data_in/simrad/ek80_test/ek80_example_data_02
./data_in/simrad/ek80_test/ek80_example_data_03
./data_in/simrad/turbeams
./data_in/simrad/y


In [5]:
all_files = index_functions.find_files(folders, ['.raw','.wcd','.all'], followlinks=True, verbose=False)
prg = tqdm(total=len(all_files), desc="Creating test data", unit="files")

for input_path in folders:
    for endings, prefix, postfix, InputFileHandler in [
        [
            ".raw",
            "raw",
            ".raw",
            simradraw.SimradRawFileHandler,
        ],
        [
            ".all",
            "all",
            ".all",
            kongsbergall.KongsbergAllFileHandler,
        ],
        [
            ".wcd",
            "wcd",
            ".wcd",
            kongsbergall.KongsbergAllFileHandler,
        ],
    ]:

        prg.set_postfix_str(f"{input_path} / {prefix}")

        prg.set_description(f"finding files")
        input_files = index_functions.find_files(input_path, endings, followlinks=True, verbose=False)

        if not input_files:
            print(f"- No {endings} files in {input_path}")
            continue

        for file in input_files:
            try:
                prg.set_postfix_str(f"{file}")

                # open and index files
                prg.set_description(f"opening files")

                fm = InputFileHandler(file, show_progress=False)

                if(len(fm.get_pings()) < N):
                    raise RuntimeError(f"Less then {N} pings in file")

                # create new file name
                file_name, file_extension = os.path.splitext(os.path.split(file)[1])
                
                prg.set_description(f"creating new file name")
                output_file = os.path.join(
                    input_path.replace(path_in, path_out), str(hash(file_name)) + file_extension
                )  # + hash of the old file name
                # print(output_file)
                os.makedirs(os.path.split(output_file)[0], exist_ok=True)

                prg.set_postfix_str(f"Writing file {output_file}")

                #get timestamps of all datagrams in Nth ping
                timestamps = [d.get_timestamp() for d in fm.get_pings()[N].file_data.datagrams()]

                # append timestamp of second PositionDatagram and second AttitudeDatagram
                if prefix in ["all", "wcd"]:                    
                    if len(fm.datagram_interface.datagrams("PositionDatagram")) > 2:
                        timestamps.append(fm.datagram_interface.datagrams("PositionDatagram")[2].get_timestamp())    
                    if len(fm.datagram_interface.datagrams("AttitudeDatagram")) > 2:                
                        timestamps.append(fm.datagram_interface.datagrams("AttitudeDatagram")[2].get_timestamp())
                else:
                    timestamps.append(fm.datagram_interface.datagrams("NME0")[1].get_timestamp())

                #compute largest timestamp
                max_timestamp = np.nanmax(timestamps)

                with open(output_file, "wb") as ofi:
                    for d in fm.datagram_interface.datagrams():
                        if d.get_timestamp() > max_timestamp:
                            break
                        ofi.write(d.to_binary())
            except Exception as e:
                print(f"--- Error with {file} ---")
                print(f"Error: {e}")
                print("---")
            prg.update(1)


Creating test data:   0%|          | 0/35 [00:00<?, ?files/s]

- No .raw files in ./data_in/kongsberg/a/c
- No .raw files in ./data_in/kongsberg/a/f/ALL
- No .wcd files in ./data_in/kongsberg/a/f/ALL
- No .raw files in ./data_in/kongsberg/a/f/WCD
- No .all files in ./data_in/kongsberg/a/f/WCD
- No .raw files in ./data_in/kongsberg/a/y
- No .raw files in ./data_in/kongsberg/e
- No .wcd files in ./data_in/kongsberg/e
- No .raw files in ./data_in/kongsberg/g


- No .raw files in ./data_in/kongsberg/he
- No .raw files in ./data_in/kongsberg/simon
- No .raw files in ./data_in/kongsberg/turbeams
- No .all files in ./data_in/simrad/ek80_test/ek80_example_data_01
- No .wcd files in ./data_in/simrad/ek80_test/ek80_example_data_01
- No .all files in ./data_in/simrad/ek80_test/ek80_example_data_02
- No .wcd files in ./data_in/simrad/ek80_test/ek80_example_data_02
- No .all files in ./data_in/simrad/ek80_test/ek80_example_data_03
- No .wcd files in ./data_in/simrad/ek80_test/ek80_example_data_03
- No .all files in ./data_in/simrad/turbeams
- No .wcd files in ./data_in/simrad/turbeams
--- Error with ./data_in/simrad/y/WBAT-1_ES70-18CD_2048_999_CAL-Phase0-D20161211-T214222-0-pp.raw ---
Error: Index [1] is >= max [0]! 
---
--- Error with ./data_in/simrad/y/tan1505-D20150424-T100829.raw ---
Error: read_sensor_configuration: No XML0 datagram found in ./data_in/simrad/y/tan1505-D20150424-T100829.raw!
---
--- Error with ./data_in/simrad/y/20151125_GASTEST_F

Warning[init_from_file]: Could not read sensor configuration from file: 
 -0: ./data_in/simrad/y/tan1505-D20150424-T100829.raw. 
 -Using empty sensor configuration as fallback. 
-Error was: read_sensor_configuration: No XML0 datagram found in ./data_in/simrad/y/tan1505-D20150424-T100829.raw!
WARNING(InputFileHandler): File [0] read incompletely. [./data_in/simrad/y/tan1905-D20190923-T010658.raw]
Stopped after 70.89 % of the file
Error message: ERROR[SimradRawDatagram]: Datagram length check failed (read). Expected: 1268, got: 0
Warning[init_from_file]: Could not read sensor configuration from file: 
 -0: ./data_in/simrad/y/tan1208-D20120617-T042023.raw. 
 -Using empty sensor configuration as fallback. 
-Error was: read_sensor_configuration: No XML0 datagram found in ./data_in/simrad/y/tan1208-D20120617-T042023.raw!
Warning[init_from_file]: Could not read sensor configuration from file: 
 -0: ./data_in/simrad/y/calV3-D20150423-T004736.raw. 
 -Using empty sensor configuration as fallback

In [13]:
input_files

[]

In [14]:
fm = simradraw.SimradRawFileHandler(input_files[4], show_progress=False)

IndexError: list index out of range

In [ ]:
print(fm)